In [1]:
import os
import subprocess
from pathlib import Path
import importlib

In [2]:
!python --version

Python 3.11.13


In [3]:
!git clone https://huggingface.co/spaces/SkalskiP/florence-sam

Cloning into 'florence-sam'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91 (from 1)
Unpacking objects: 100% (91/91), 17.10 KiB | 416.00 KiB/s, done.
Filtering content: 100% (7/7), 1.46 GiB | 63.15 MiB/s, done.


In [4]:
%cd florence-sam

/content/florence-sam


| Library        | Version     |
|----------------|-------------|
| tqdm           | 4.67.1      |
| einops         | 0.8.1       |
| spaces         | 0.39.0      |
| timm           | 1.0.19      |
| transformers   | 4.54.1      |
| samv2          | 0.0.4       |
| gradio         | 5.39.0      |
| supervision    | 0.26.1      |
| opencv-python  | 4.12.0.88   |
| pytest         | 8.4.1       |


In [5]:
with open("requirements.txt", "w") as f:
    f.write("""
tqdm==4.67.1
einops==0.8.1
spaces==0.39.0
timm==1.0.19
transformers==4.54.1
samv2==0.0.4
gradio==5.39.0
supervision==0.26.1
opencv-python==4.12.0.88
pytest==8.4.1
""".strip())

In [6]:
!sed -i '/import spaces/d' app.py
!sed -i '/@spaces.GPU/d' app.py
!sed -i 's/imports.remove("flash_attn")/if "flash_attn" in imports: imports.remove("flash_attn")/' "utils/florence.py"

In [7]:
!pip install -r requirements.txt -q

In [8]:
!sed -i '/launch(/s/launch(\(.*\))/launch(share=True, \1)/' app.py

In [9]:
!sed -i '/generated_ids = model.generate(/a\        use_cache=False,' "utils/florence.py"

In [10]:
app_path = "app.py"

with open(app_path, "r") as f:
    lines = f.readlines()

patch = [
    "\n",
    "# Florence patch to fix SDPA error\n",
    "import subprocess\n",
    "try:\n",
    "    modeling_path = subprocess.check_output([\n",
    "        \"find\", os.path.expanduser(\"~/.cache/huggingface/\"),\n",
    "        \"-type\", \"f\", \"-name\", \"modeling_florence2.py\"\n",
    "    ], text=True).strip()\n",
    "    if os.path.exists(modeling_path):\n",
    "        with open(modeling_path, \"r\") as f:\n",
    "            content = f.read()\n",
    "        if \"_supports_sdpa = False\" not in content:\n",
    "            subprocess.run([\n",
    "                \"sed\", \"-i\",\n",
    "                \"/class Florence2ForConditionalGeneration/a\\\\        _supports_sdpa = False\",\n",
    "                modeling_path\n",
    "            ])\n",
    "            print(\"Patched _supports_sdpa = False into:\", modeling_path)\n",
    "        else:\n",
    "            print(\"Already patched:\", modeling_path)\n",
    "    else:\n",
    "        print(\"Florence file not found.\")\n",
    "except Exception as e:\n",
    "    print(\"Error during Florence patching:\", e)\n"
]

injection_point = None
for i, line in enumerate(lines):
    if "FLORENCE_MODEL, FLORENCE_PROCESSOR = load_florence_model(device=DEVICE)" in line:
        injection_point = i + 1
        break

if injection_point:
    new_lines = lines[:injection_point] + patch + lines[injection_point:]
    with open(app_path, "w") as f:
        f.writelines(new_lines)
    print("Patch successfully injected into app.py")
else:
    print("Could not find injection point.")

Patch successfully injected into app.py


In [11]:
!python app.py

2025-08-07 14:38:53.719853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754577533.750063    7734 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754577533.756498    7734 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754577533.772516    7734 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754577533.772549    7734 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754577533.772553    7734 computation_placer.cc:177] computation placer alr